In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
from flask import Flask, render_template, request
import requests
import pickle
import keras
import tensorflow as tf
from flask import jsonify
tf.keras.backend.set_floatx('float64')
from tensorflow.keras import layers
tf.random.set_seed(42)
import numpy as np
app=Flask(__name__,template_folder='template')

@app.route('/',methods=['GET'])
def Home():
    return render_template('index.html')

@app.route("/predict", methods=['POST'])
def predict():
    if request.method == 'POST':
        # How many timesteps to predict into the future?
        INTO_FUTURE = int(request.form['INTO_FUTURE']) 
        data = pd.read_csv('data.csv', index_col=["Date"])
        da = pd.read_csv('data.csv')
        da['Date'] = pd.to_datetime(da['Date'])
        df = da[['PM2.5', 'Date']]
        df.set_index("Date", inplace = True)

        timesteps = df.index.to_numpy()
        prices = df["PM2.5"].to_numpy()

        # Create train and test splits the right way for time series data
        split_size = int(0.8 * len(prices))
        X_train, y_train = timesteps[:split_size], prices[:split_size]
        X_test, y_test = timesteps[split_size:], prices[split_size:]

        HORIZON = 1 # predict 1 step at a time
        WINDOW_SIZE = 7 # use a week worth of timesteps to predict the horizon

        def get_labelled_windows(x, horizon=1):
            return x[:, :-horizon], x[:, -horizon:]

        # Create function to view NumPy arrays as windows 
        def make_windows(x, window_size=7, horizon=1):
            window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
            window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).T
            windowed_array = x[window_indexes]
            windows, labels = get_labelled_windows(windowed_array, horizon=horizon)
            return windows, labels

        full_windows, full_labels = make_windows(prices, window_size=WINDOW_SIZE, horizon=HORIZON)

        # Make the train/test splits
        def make_train_test_splits(windows, labels, test_split=0.2):
            split_size = int(len(windows) * (1-test_split)) 
            train_windows = windows[:split_size]
            train_labels = labels[:split_size]
            test_windows = windows[split_size:]
            test_labels = labels[split_size:]
            return train_windows, test_windows, train_labels, test_labels

        train_windows, test_windows, train_labels, test_labels = make_train_test_splits(full_windows, full_labels)


        # Create a function to implement a ModelCheckpoint callback with a specific filename 
        def create_model_checkpoint(model_name, save_path="model_experiments"):
            return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                                    verbose=0, # only output a limited amount of text
                                                    save_best_only=True) # save only the best model to file

        x = tf.constant(train_windows[0])
        expand_dims_layer = layers.Lambda(lambda x: tf.expand_dims(x, axis=1))


        # Create model
        model_4 = tf.keras.Sequential([
          layers.Lambda(lambda x: tf.expand_dims(x, axis=1)),
          layers.Conv1D(filters=128, kernel_size=5, padding="causal", activation="relu"),
          layers.Dense(HORIZON)
        ], name="model_4_conv1D")

        # Compile model
        model_4.compile(loss="mae",
                        optimizer=tf.keras.optimizers.Adam())

        # Fit model
        model_4.fit(train_windows,
                    train_labels,
                    batch_size=128, 
                    epochs=100,
                    verbose=0,
                    validation_data=(test_windows, test_labels),
                    callbacks=[create_model_checkpoint(model_name=model_4.name)])

        # 1. Create function to make predictions into the future
        def make_future_forecast(values, model, into_future, window_size=WINDOW_SIZE) -> list:
            future_forecast = []
            last_window = values[-WINDOW_SIZE:] 
            for _ in range(into_future):
                future_pred = model.predict(tf.expand_dims(last_window, axis=0))
                future_forecast.append(tf.squeeze(future_pred).numpy())
                last_window = np.append(last_window, future_pred)[-WINDOW_SIZE:]

            return future_forecast


        y_all = df["PM2.5"].to_numpy()
        future_forecast = make_future_forecast(values=y_all,
                                               model=model_4,
                                               into_future=INTO_FUTURE,
                                               window_size=WINDOW_SIZE)


        def get_future_dates(start_date, into_future, offset=1):
            start_date = start_date + np.timedelta64(offset, "D") 
            end_date = start_date + np.timedelta64(into_future, "D") 
            return np.arange(start_date, end_date, dtype="datetime64[D]")


        last_timestep = df.index[-1]
        next_time_steps = get_future_dates(start_date=last_timestep, 
                                           into_future=INTO_FUTURE)
        btc = df['PM2.5'].to_numpy()


        ft = list(zip(next_time_steps, future_forecast))
        a = pd.DataFrame(ft)
        col = [['Date', 'PM2.5_level']]
        a.columns = col
        a.to_html('C:/Users/User/Desktop/aqi/template/model.html')
        return render_template('model.html')
    
if __name__ == "__main__":
    app.run(debug=True, use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jul/2021 15:45:02] "GET / HTTP/1.1" 200 -


INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets written to: model_experiments\model_4_conv1D\assets
INFO:tensorflow:Assets wr

127.0.0.1 - - [03/Jul/2021 15:46:25] "POST /predict HTTP/1.1" 200 -
